In [ ]:
"""My analysis of GitHub Users of Austin,Texas"""

In [2]:
import pandas as pd
import statsmodels.api as sm
from collections import Counter
users=pd.read_csv('users.csv')

In [10]:
#top 5 users by followers
aus=users[users['location'].str.contains('Austin', case=False)]
top=aus.nlargest(5, 'followers')
ans=','.join(top['login'])
ans

'getify,benawad,steveklabnik,cloudflare,jbogard'

In [9]:
#earliest 5 registered users
users['created_at']=pd.to_datetime(users['created_at'])
aus=users[users['location'].str.contains('Austin', case=False)]
old=aus.nsmallest(5, 'created_at')
ans=','.join(old['login'])
ans

'jnewland,joshknowles,hassox,jicksta,dan'

In [4]:
#top 3 most popular licenses
repo=pd.read_csv('repositories.csv')
lic=repo['license_name'].value_counts()
top=lic.nlargest(3).index.tolist()
ans=','.join(top)
ans

'mit,apache-2.0,other'

In [5]:
#most common company
emp=users['company'].str.upper().str.strip('@') 
top=emp.value_counts()
org=top.idxmax()
org

'GOOGLE'

In [6]:
#most popular programming language
lng=repo['language'].value_counts()
top=lng.idxmax()
top

'JavaScript'

In [7]:
#second most popular language post 2020
recentusers=users[users['created_at']>'2020-01-01']
recentrepos=repo[repo['login'].isin(recentusers['login'])]
secondlang=recentrepos['language'].value_counts().nlargest(2).index[1]
secondlang

'HTML'

In [8]:
#language with highest average stars per repository
avg=repo.groupby('language')['stargazers_count'].mean()
top=avg.idxmax()
top

'Fennel'

In [9]:
#top 5 users by leader strength
ldr=users['followers']/(1+users['following'])
users['ldr']=ldr
top=users.nlargest(5, 'ldr')['login']
ans=','.join(top)
ans

'getify,cloudflare,benawad,oracle,ContinuumIO'

In [10]:
#correlation between followers and repositories
atx=users[users['location'].str.contains('Austin', case=False)]
cor=atx['followers'].corr(atx['public_repos'])
ans=round(cor, 3)
ans

0.151

In [11]:
#impact of repositories on followers
x=users['public_repos']
y=users['followers']
x=sm.add_constant(x)
mod=sm.OLS(y, x).fit()
bet=mod.params['public_repos']
ans=round(bet, 3)
ans

4.104

In [12]:
#correlation between projects and wikis
projwcorr= repo['has_projects'].corr(repo['has_wiki'])
round(projwcorr, 3)

0.274

In [13]:
#hireable users and following
hire=users[users['hireable']==True]['following'].mean()-users[users['hireable']==False]['following'].mean()
round(hire, 3)

109.041

In [14]:
#impact of bio length on followers
users['bw']=users['bio'].fillna('').str.split().apply(len)
bx=users[users['bw']>0]['bw']
by=users[users['bw']>0]['followers']
bx=sm.add_constant(bx)
bfmodel=sm.OLS(by, bx).fit()
bfslope= bfmodel.params['bw']
round(bfslope, 3)

7.785

In [15]:
#most prolific weekend developers
repo['created_at']=pd.to_datetime(repo['created_at'])
repo['is_weekend']=repo['created_at'].dt.weekday>=5
weekndrepos = repo[repo['is_weekend']].groupby('login').size().nlargest(5).index
','.join(weekndrepos)

'FellowTraveler,realityexpander,OR13,PaulBratslavsky,skeptycal'

In [16]:
#email sharing and hireability
emailsharehireable=users[users['hireable']==True]['email'].notna().mean()
emailsharenothireable=users[users['hireable'] == False]['email'].notna().mean()
emailsharediff=round(emailsharehireable - emailsharenothireable, 3)
emailsharediff

0.022

In [17]:
#most common surname
snct=Counter()
for nm in users['name'].fillna('').str.strip():
    if nm:
        sn=nm.split()[-1]
        snct[sn]+=1
if snct:
    mxct=max(snct.values())
    cmnsns=[sn for sn, ct in snct.items() if ct==mxct]
    cmnsns.sort()
    print(','.join(cmnsns))
else:
    print("No names found.")

Labs,Moore,Smith


In [18]:
"""Thankyou"""

'Thankyou'